In [ ]:
#CNN1v-LSTM

import pandas as pd
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM,GRU
import sklearn
from sklearn import metrics
import time

#Input data
#indata=pd.read_csv("../input/DGA_test.csv")
indata=pd.read_csv(r'E:\Data_test\DGA.csv',header=None)

X=indata.iloc[:,0]
labels=indata.iloc[:,1]

# Generate a dictionary of valid characters
valid_chars = {x:idx+1 for idx, x in enumerate(set(''.join(X)))}

max_features = len(valid_chars) + 1
maxlen = np.max([len(x) for x in X])

# Convert characters to int and pad
X = [[valid_chars[y] for y in x] for x in X]
X = sequence.pad_sequences(X, maxlen=maxlen)
y=np.array(labels)

train_x=X[0:20000]
train_y=y[0:20000]
test_x=X[20000:30000]
test_y=y[20000:30000]

#print(max_features)
time_start=time.time()

model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(layers.Conv1D(32,5,activation='relu'))
model.add(layers.MaxPooling1D(3))
model.add(layers.Conv1D(32,5,activation='relu'))
#model.add(layers.GlobalMaxPooling1D())
model.add(Dropout(0.5))
model.add(LSTM(128))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop')

model.fit(train_x,train_y,batch_size=20,nb_epoch=1)

t_test_probs = model.predict(test_x)
t_train_probs = model.predict(train_x)
t_test_auc = sklearn.metrics.roc_auc_score(test_y, t_test_probs)
t_train_auc = sklearn.metrics.roc_auc_score(train_y, t_train_probs)

time_end=time.time()

print("train AUC:",round(t_train_auc,2))
print("test AUC:",round(t_test_auc,2))
print("time:",round(time_end-time_start,2))